In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import lightgbm as lgb 
import datetime
import math
import gc 


In [2]:
from subprocess import check_output
print(check_output(["ls"]).decode("utf8"))

Recomender Engine - Songs.html
Recomender Engine - Songs.ipynb
members.csv
sample_submission.csv
song_extra_info.csv
songs.csv
submission_lgbm_avg.csv
test.csv
train.csv



In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
members = pd.read_csv("members.csv",  parse_dates=['registration_init_time','expiration_date'])
songs = pd.read_csv("songs.csv")
song_extra_info = pd.read_csv("song_extra_info.csv")


In [4]:
Final = train.append(test)
Final = Final.merge(songs , on = "song_id", how = "left")

members['membership_days'] = members['expiration_date'].subtract(members['registration_init_time']).dt.days.astype(int)
members['reg_year'] = members['registration_init_time'].dt.year
members['reg_mth'] = members['registration_init_time'].dt.month
members['reg_day'] = members['registration_init_time'].dt.day
members['exp_year'] = members['expiration_date'].dt.year
members['exp_mth'] = members['expiration_date'].dt.month
members['exp_day'] = members['expiration_date'].dt.day


members = members.drop(['registration_init_time', "expiration_date"], 1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [5]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan
    
song_extra_info['year'] = song_extra_info['isrc'].apply(isrc_to_year)

In [20]:
Final = Final.merge(members, on = "msno", how = "left")
Final = Final.merge(song_extra_info, on = "song_id", how = "left")

Final.song_length.fillna(Final.song_length.median(), inplace = True)
Final.song_id = Final.song_id.astype('category')

del members, songs, song_extra_info

In [21]:
Final.isnull().sum()

id                    7377418
msno                        0
song_id                     0
source_screen_name     577687
source_system_tab       33291
source_type             28836
target                2556790
song_length                 0
genre_ids              160565
artist_name               139
composer              2295010
lyricist              4403542
language                  192
city_x                      0
bd_x                        0
gender_x              4013703
registered_via_x            0
membership_days_x           0
reg_year_x                  0
reg_mth_x                   0
reg_day_x                   0
exp_year_x                  0
exp_mth_x                   0
exp_day_x                   0
name_x                   2235
isrc_x                 774501
year_x                 774501
city_y                      0
bd_y                        0
gender_y              4013703
                       ...   
isrc_y                 774501
year_y                 774501
city_x    

In [22]:
def genre_id_count(x):
    if x == 'NaN':
        return 0
    else:
        return x.count('|') + 1

Final['genre_ids_cnt'] = Final['genre_ids'].astype('category').apply(genre_id_count)

def lyricist_count(x):
    if x == 'NaN':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1
    return sum(map(x.count, ['|', '/', '\\', ';']))
Final['lyricist_cnt'] = Final['lyricist'].astype('category').apply(lyricist_count)

def composer_count(x):
    if x == 'NaN':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1
Final['composer_cnt'] = Final['composer'].astype('category').apply(composer_count)

def is_featured(x):
    if 'feat' in str(x) :
        return 1
    return 0
Final['is_featured'] = Final['artist_name'].apply(is_featured).astype(np.int8)

def artist_count(x):
    if x == 'NaN':
        return 0
    else:
        return x.count('and') + x.count(',') + x.count('feat') + x.count('&')

Final['artist_count'] = Final['artist_name'].astype('category').apply(artist_count).astype(np.int8)

# if artist is same as composer
Final['artist_composer'] = (Final['artist_name'] == Final['composer']).astype(np.int8)

# if artist, lyricist and composer are all three same
Final['artist_composer_lyricist'] = ((Final['artist_name'] == Final['composer']) & (Final['artist_name'] == Final['lyricist']) & (Final['composer'] == Final['lyricist'])).astype(np.int8)


# is song language 17 or 45. 
def song_lang_boolean(x):
    if '17.0' in str(x) or '45.0' in str(x):
        return 1
    return 0

Final['song_lang_boolean'] = Final['language'].apply(song_lang_boolean).astype(np.int8)


_mean_song_length = np.mean(Final['song_length'])
def smaller_song(x):
    if x < _mean_song_length:
        return 1
    return 0

Final['smaller_song'] = Final['song_length'].apply(smaller_song).astype(np.int8)

# number of times a song has been played before
_dict_count_song_played_final = {k: v for k, v in Final['song_id'].value_counts().iteritems()}

def count_song_played(x):
    try:
        return _dict_count_song_played_final[x]
    except KeyError:
            return 0
    

Final['count_song_played'] = Final['song_id'].apply(count_song_played).astype(np.int64)


# number of times the artist has been played
_dict_count_artist_played_final = {k: v for k, v in Final['artist_name'].value_counts().iteritems()}
def count_artist_played(x):
    try:
        return _dict_count_artist_played_final[x]
    except KeyError:
            return 0

Final['count_artist_played'] = Final['artist_name'].apply(count_artist_played).astype(np.int64)



In [26]:
for col in Final.columns:
    Final[col]

In [25]:
print("Train test and validation sets")

for col in Final.columns:
    if Final[col].dtype == object:
        Final[col] = Final[col].astype('category')

train = Final[-Final['target'].isnull()]
test = Final[Final['target'].isnull()]

X_train = train.drop(['target'], axis=1)
y_train = train['target'].values

X_test = test.drop(['id'], axis=1)
ids = test['id'].values


del train, test; gc.collect();

d_train_final = lgb.Dataset(X_train, y_train)
watchlist_final = lgb.Dataset(X_train, y_train)
print('Processed data...')

Train test and validation sets


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
params1 = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }

%time model_f1 = lgb.train(params1, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=5)

params2 = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'dart',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }

%time model_f2 = lgb.train(params2, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=5)

In [ ]:
print('Making predictions')
p_test_1 = model_f1.predict(X_test)
p_test_2 = model_f2.predict(X_test)
p_test_avg = np.mean([p_test_1, p_test_2], axis = 0)


print('Done making predictions')

In [ ]:
print ('Saving predictions Model of gbdt')

subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test_avg
subm.to_csv('submission_lgbm_avg.csv', index = False, float_format = '%.5f')

print('Done!')